In [40]:
import torch
import torchvision.transforms as transforms
import torchvision.models as models
from PIL import Image
from ultralytics import YOLO

# ✅ Use GPU if available, otherwise use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



In [1]:
# ✅ Path to trained YOLO model
# yolo_model_path = "E:/anul 4/LICENTA_NISTOR_IOAN_GABRIEL/runs/detect/train/weights/best.pt"
yolo_model_path = "E:/anul 4/LICENTA INVATARE/runs/detect_roi/train3/weights/best.pt"

yolo_model = YOLO(yolo_model_path)
yolo_model.eval()

# ✅ Path to trained ResNet-50 model
num_classes = 30 
resnet = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V2)
num_ftrs = resnet.fc.in_features
resnet.fc = torch.nn.Linear(num_ftrs, num_classes)
resnet.load_state_dict(torch.load("E:/anul 4/LICENTA INVATARE/runs/detect_roi/train_resnet_fruits/FineTunedResnet/resnet_fruits_epoch_1.pth", map_location=device))
resnet = resnet.to(device)
resnet.eval()

# ✅ ResNet Transform
resnet_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# asta poate
#resnet_transform = transforms.Compose([
 #   transforms.Resize(224),
  #  transforms.Pad(padding),
   # transforms.CenterCrop(224),
   # transforms.ToTensor(),
    #transforms.Normalize(mean=[0.485, 0.456, 0.406],
     #                    std=[0.229, 0.224, 0.225]),
#])


NameError: name 'YOLO' is not defined

In [42]:
def detect_and_crop(image_path, model_yolo):
    """Detect objects in an image and return a list of cropped images."""
    
    image = Image.open(image_path).convert("RGB")
    
    # ✅ Run YOLO on the image
    results = model_yolo(image_path)
    cropped_images = []

    for result in results:
        for bbox, cls in zip(result.boxes.xywh, result.boxes.cls):
            x_c, y_c, w, h = bbox
            x_min = int((x_c - w / 2))
            y_min = int((y_c - h / 2))
            x_max = int((x_c + w / 2))
            y_max = int((y_c + h / 2))

            # ✅ Crop the object from the image
            cropped_img = image.crop((x_min, y_min, x_max, y_max))
            cropped_images.append(cropped_img)
            
    if(len(cropped_images) > 0):
        print(f"✅ Detected {len(cropped_images)} objects.")
        return cropped_images 
    else:
        print("⚠️ No detection! Use the entire image.")
        return image 


In [43]:
def classify_with_resnet(image, model_resnet, class_names):
    """Receive an image and return the predicted class."""
    
    image = resnet_transform(image).unsqueeze(0).to(device)
    
    with torch.no_grad():
        output = model_resnet(image)
        predicted_class = torch.argmax(output, dim=1).item()
    
    return class_names[predicted_class]


In [47]:
# ✅ Set the path to the test image
test_image_path = "E:/anul 4/LICENTA_NISTOR_IOAN_GABRIEL/datasets/food.v1i.yolov11/test3/14.jpg"

cropped_images = detect_and_crop(test_image_path, yolo_model)

for i, cropped_image in enumerate(cropped_images):
    predicted_label = classify_with_resnet(cropped_image, resnet, [str(i) for i in range(num_classes)])
    print(f"🎯 Predicted class for object {i+1}: {predicted_label}")



image 1/1 E:\anul 4\LICENTA_NISTOR_IOAN_GABRIEL\datasets\food.v1i.yolov11\test3\14.jpg: 640x480 4 rois, 2223.1ms
Speed: 100.7ms preprocess, 2223.1ms inference, 52.0ms postprocess per image at shape (1, 3, 640, 480)
✅ Detected 4 objects.
🎯 Predicted class for object 1: 21
🎯 Predicted class for object 2: 4
🎯 Predicted class for object 3: 23
🎯 Predicted class for object 4: 2


In [45]:
#!yolo val model="E:/anul 4/LICENTA INVATARE/runs/detect/train2/weights/best.pt" data="E:/anul #4/licenta/dataset/FruitsAndVegetablesFinal/data.yaml"         
